# 字典和集合

集合和字典很像，只是不包含value，是一堆key 的组合。

使用字典和集合的优势:
- 查询和插入的时间复杂度均为$O(1)$(使用底层数据结构: 开放地址散列表)。

局限性是：
- 通常会使用更大的内存
- 实际速度取决于散列函数(虽然复杂度为$O(1)$)

## 1. 字典和集合基础

### 1.1 字典

下面，我们看如何使用一个列表来实现电话簿查询功能。

#### 列表版本

In [1]:
phonebook = [
    ("John Doe", "555-555-5555"),
    ("Albert Einstein", "212-555-5555"),
]

In [5]:
def find_phonenumber(phonebook, input_name):
    for name, phone in phonebook:
        if name == input_name:
            return phone
    return None

In [6]:
find_phonenumber(phonebook, "John Doe")  # 找到

'555-555-5555'

In [7]:
find_phonenumber(phonebook, "Obama")  # 没找到

#### 字典版本

In [8]:
phonebook = {
    "John Doe": "555-555-5555",
    "Albert Einstein" : "212-555-5555",
}

In [9]:
phonebook["John Doe"]  # 找到，返回结果

'555-555-5555'

In [10]:
phonebook["Obama"]  # 找不到，抛出KeyError

KeyError: 'Obama'

#### 注意

一般在字典查询是要使用try...except... 来捕捉异常。

### 1.2 集合

In [16]:
phonebook = [
("John Doe", "555-555-5555"),
("Albert Einstein", "212-555-5555"),
("John Murphey", "202-555-5555"),
("Albert Rutherford", "647-555-5555"),
("Elaine Bodian", "301-555-5555"),
]

下面我们写两个函数来计算一个列表中的unique first name. `list_unique_names`使用列表，`set_unique_names` 使用集合。

我们可以看出，上面phonebook 中有5个人，其中有两个人的first name 是John，两个人叫Albert。所以unique first name 应该是3个。

In [23]:
def list_unique_names(phonebook):
    unique_names = []  # 使用列表存储unique names
    for name, phonenumber in phonebook:  # 复杂度 O(n)
        first_name, last_name = name.split(" ", 1)
        for unique in unique_names: #  复杂度 O(n) 
            if unique == first_name: 
                break
        else:  # else 和for loop 一起使用，意味着没有loop 中所有的if 都没有执行，然后执行loop 之外的else. 
            unique_names.append(first_name)
    return len(unique_names)

In [14]:
def set_unique_names(phonebook):
    unique_names = set()  # 使用集合存储unique names
    for name, phonenumber in phonebook:  # 复杂度 O(n)
        first_name, last_name = name.split(" ", 1)
        unique_names.add(first_name) #  无需遍历unique_names 复杂度O(1)
    return len(unique_names)

In [21]:
list_unique_names(phonebook)

3

In [17]:
set_unique_names(phonebook)

3

`list_unique_names` 内部循环的复杂度为$O(n)$, 外部循环的复杂度也是$O(n)$, 所以整个方法的复杂度为$O(n^2)$.

`set_unique_names` 内部循环的复杂度为$O(1)$, 外部循环的复杂度是$O(n)$, 所以整个方法的复杂度为$O(n)$.

对于有10,000 entries 和 7,422 unique first name 的phonebook，我们使用`list_unique_names` 的时间为2.5s左右，使用`set_unique_names` 的时间为9.5ms。

#### 注意

我们在`list_unique_names` 函数中在for loop 之后使用了else. More explaination can be find [here](https://stackoverflow.com/questions/9979970/why-does-python-use-else-after-for-and-while-loops).

A common construct is to run a loop until something is found and then to break out of the loop. The problem is that if I break out of the loop or the loop ends I need to determine which case happened. One method is to create a flag or store variable that will let me do a second test to see how the loop was exited.

For example assume that I need to search through a list and process each item until a flag item is found and then stop processing. If the flag item is missing then an exception needs to be raised.

## 2. 字典和集合如何工作

### 2.1 插入

插入数据的位置取决于两个因素：
- 键的散列值
    - 计算key 的散列值 
    - 通过掩码来得到一个有效的数组索引。掩码用于截断一个数字，通常意味着取这段数字的最后几位。假如我们数组长度为10，我们可以使用一位十进制的掩码，假如我们数组长度为1024，我们可以使用10位2进制的掩码。
- 该值如何跟其他对象比较
    - 如果该索引对应的内存单元为空，则可以存储；
    - 如果该索引对应的内存单元不为空，我们需要判断内存单元中的值和我们要插入的值(value)是否相同(`cmp`)：
        - 如果相同，说明该值已保存，则返回
        - 如果不相同，我们需要计算出一个新的索引，这一方法称为嗅探(probing).

#### 插入示例
假设我们要用字典存储以下数据：

```python
data = {
    "Rome": 4,
    "San Francisco": 3,
    "New York": 5,
    "Barcelona": 2,
}
```
我们首先定义hash 函数和掩码：
- hash 函数: 对key 的第一个字符应用ord 函数，返回一个整数。`ord(key[0])`
- 掩码: 对于一个拥有四个元素的字典，将分配8个地址单元(最小)，因此掩码位数为三位，即0b111($2^3=8$).

接下来，我们演示以下数据插入的流程。

1. 例如下图，我们首先插入数据: {key = Rome, value = 4}. `ord('R')` 的结果是82，二进制为'0b1010010'. 然后3位掩码，即取最低3位作为数组索引(0b010)，即2. 所以该条数据将被插入数组第3个位置(数组的index 从0开始)。

2. 接下来，我们插入数据{key = San Francisco, value = 3}. `ord('S')` 的结果是83，二进制为'0b1010011'. 因此索引为3(0b011)，即插入数组第三个位置。

3. 接下来，我们插入数据{key = New York, value = 5}. `ord('N')` 的结果是78，二进制为'0b1001110'. 因此索引为6(0b110)，即插入数组第7个位置。

4. 最后，我们插入数据{key = Barcelona, value = 2}. `ord('B')` 的结果是66，'0b1000010'. 因此索引为2(0b010)，即插入数组第3个位置。这时，我们发现发生了碰撞。

5. 然后我们进行嗅探，这里我们使用线性嗅探(linear probing). 假设我们的嗅探是：i = (5 * i + 1) & mask. i 是初始的散列值，这里位66. 所以下一个嗅探位置数5*66+1 =341. 二进制为: '0b101010101'.通过掩码后所以为5(0b101),即插入数组第6个位置。这时，我们看出第六个位置为空，则准许插入。

整个过程如下图所示。

<img src="figures/insert_hash.png" alt="drawing" width="400"/>

⚠️hash 表存储的时候会存储key 和value，用于查询。

In [30]:
bin(ord('R'))

'0b1010010'

In [31]:
bin(ord('S'))

'0b1010011'

In [32]:
bin(ord('N'))

'0b1001110'

In [33]:
bin(ord('B'))

'0b1000010'

In [37]:
bin(5*66 + 11)

'0b101010101'

In [38]:
5*66 + 11

341

### 2.2 查找

查找的过程与插入的过程相似，我们只对产生碰撞的情况做一个说明。

假设我们需要查询key = Barcelona 的value. 
1. 首先`ord('B')` 的结果是66，'0b1000010'. 
2. 通过掩码计算，索引为2(0b010)，即数组第3个位置。
3. 我们发现，存在第三个位置的key 是Rome，这时，我们知道发生了碰撞。
4. 然后我们进行嗅探，这里同样使用线性嗅探(linear probing). i = (5 * i + 1) & mask. 所以下一个嗅探位置数5*66+1 =341. 二进制为: '0b101010101'.通过掩码后所以为5(0b101),即插入数组第6个位置。
5. 这时，我们发现第六个位置为的key 是Barcelona，则返回其值2。

如果找到的是一个空，我们则可以判定该元素不在字典中。

### 2.3 删除

删除一个元素，可能导致的问题是查询出错。

例如，我们删除了上面三列表中的Rome 数据。则在查询是，我们通过散列函数+掩码找到第三个位置单元，我们发现这是一个空的单元，因此我们判定当前字典并不包含Barcelona。会导致错误。

实际上Python 会在删除的内存单元写入一个特殊值，来表示该桶虽空，但是其后可能还有别的因碰撞二插入的值，需要继续嗅探。

### 2.4 改变大小(resize)

上面已经提到，初始化一个字典的最小长度是8. 
- 每次改变大小，会扩容到以前的4倍
- 超过50000 个元素后，每次扩容到原先的2倍

即：8, 32, 128, 512, 8192, 32768, 131072, 262144, ...

研究显示，一个不超过2/3 的表在具有最佳空间节约的同时依然具有不错的散列碰撞避免率。

## 3. 散列函数和熵

### 3.1 Python 对象的散列表

`__hash__` and `__cmp__`

- int/float: bit value
- tuple/string: content
- list: not support hashing - values can change.
- User-defined classes: 默认的散列函数返回内存中的位置(id function), 默认的比较是否相等也是比较地址。

这样的默认，在有时会出现歧义，例如：

In [48]:
class Point():
    def __init__(self, x, y):
        self.x, self.y = x, y

In [49]:
p1 = Point(1,1)
p2 = Point(1,1)

In [50]:
points = set([p1, p2])

In [51]:
len(points)

2

In [52]:
p1 == p2

False

我们可以看出，p1 和p2 其实是一个点，因为是两个对象，在内存中占有不同的内存地址，因此使用默认的方法会判断成两个点。未解决这个问题，我们可以重写`__hash__` 和`__eq__`方法。

In [53]:
class MyPoint():
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __hash__(self):
        return hash((self.x, self.y))  # 返回一个tuple 的hash值
    
    def __eq__(self, another_point):
        return self.x == another_point.x and self.y == another_point.y

In [54]:
p1 = MyPoint(1,1)
p2 = MyPoint(1,1)

In [55]:
p1 == p2

True

In [56]:
points = set([p1, p2])

In [58]:
len(points)  # 因为p1 p2 被判断成一个点，因此set 长度为1

1

### 3.2 散列函数的熵

我们可以看出，无论是插入还是查询，碰撞发生的太多，会频繁的嗅探其他索引值，因此会极大的影响效率。如果一个字典中所有的key 都碰撞，则搜索字典和搜索列表的复杂度一样$O(n)$.

通常，我们希望数据可以均匀的分布在存储空间中(依托于一个好的散列函数)。数据的均匀程度称为负载因素(load factor), 它跟散列函数的熵有关。

衡量散列函数分布均匀程度的标准被称为散列函数的熵。

$$S = -\Sigma p(i)log(p(i))$$

p(i) 是散列函数给出值为i 的概率(这通常取决于你的应用场景)。所以完美的散列函数是根据场景决定的。如果所有值的概率相同，则熵最小，分布最均匀，我们称这样的散列函数为完美散列函数。

#### 确定掩码长度

如果字典的大小为N，则我们为保证性能，我们需要保证N 为约2/3 的空间，那我们需要N*3/2 大小的空间。例如N=1039，则我们需要1559个存储空间。也就是说，我们的字典需要2048个存储位置。我们需要的掩码是bin(2018-1) = '0b11111111111'.

In [59]:
bin(2048 - 1)

'0b11111111111'

#### good v.s. bad

下面我们来展示一个好的哈希函数和一个坏的哈希函数的区别。

- bad one: 对于所有输入都返回相同的值, 即所有元素都会碰撞
- good one: 

In [60]:
class BadHash(str):
    def __hash__(self):
        return 42

class GoodHash(str):
    def __hash__(self):
        """
        This is a slightly optimized version of twoletter_hash
        """
        return ord(self[1]) + 26 * ord(self[0]) - 2619

In [62]:
import string

baddict = set()
gooddict = set()

for i in string.ascii_lowercase:
    for j in string.ascii_lowercase:
        key = i + j
        baddict.add(BadHash(key))
        gooddict.add(GoodHash(key))

In [77]:
import time

t1 = time.time()
for i in range(1000000):
    key = BadHash('zz')
    key in baddict
t2 = time.time()
print(t2-t1)

17.177631378173828


In [74]:
import time

t1 = time.time()
for i in range(1000000):
    key = GoodHash('zz')
    key in gooddict
t2 = time.time()
print(t2-t1)

0.035019874572753906


可以看到，频繁的hash 碰撞，导致频繁嗅探，会严重影响效率。

## 4. 字典和命名空间

Python 命名空间的管理使用了字典。

当Python 访问一个标量，函数或模块是：
- 首先找`locals()`数组 -- 优化过，很快
- 没找到，查找`globals()`字典
- 最后，搜索`__builtin__` 对象，其内置了一个`locals()`字典

In [79]:
import math
from math import sin

def test1(x):  # 381ns
    return math.sin(x)

def test2(x):
    return sin(x)  # 311ns

def test3(x, sin=math.sin):  # 306ns
    return sin(x) 

In [83]:
%timeit test1(123456)

212 ns ± 6.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [85]:
%timeit test2(123456)

195 ns ± 3.18 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [86]:
%timeit test3(123456)

180 ns ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


我们可以看到第三种方法最快。原因是，第三种方法没有做字典查询。我们通过dis 模块来解析命名空间的解析。

In [88]:
import dis 

dis.dis(test1)

'''
  5           0 LOAD_GLOBAL              0 (math)  -- dict 查找
              2 LOAD_ATTR                1 (sin)   -- dict 查找
              4 LOAD_FAST                0 (x)     -- local 查找
              6 CALL_FUNCTION            1
              8 RETURN_VALUE
'''

  5           0 LOAD_GLOBAL              0 (math)
              2 LOAD_ATTR                1 (sin)
              4 LOAD_FAST                0 (x)
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


In [89]:
dis.dis(test2)

'''
  8           0 LOAD_GLOBAL              0 (sin)  -- dict 查找
              2 LOAD_FAST                0 (x)    -- local 查找
              4 CALL_FUNCTION            1
              6 RETURN_VALUE
'''

  8           0 LOAD_GLOBAL              0 (sin)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE


In [90]:
dis.dis(test3)

'''
 11           0 LOAD_FAST                1 (sin)  -- local 查找
              2 LOAD_FAST                0 (x)    -- local 查找
              4 CALL_FUNCTION            1
              6 RETURN_VALUE
'''

 11           0 LOAD_FAST                1 (sin)
              2 LOAD_FAST                0 (x)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE


通常的一个技巧是，再循环前设置一个本地变量来保存一个函数的全局引用。

In [95]:
def tight_loop_slow(iterations):
    """
    >>> %timeit tight_loop_slow(10000000)
    1 loops, best of 3: 2.21 s per loop
    """
    result = 0
    for i in range(iterations):
        # this call to sin requires a global lookup
        result += sin(i)

In [96]:
def tight_loop_fast(iterations):
    """
    >>> %timeit tight_loop_fast(10000000)
    1 loops, best of 3: 2.02 s per loop
    """
    result = 0
    local_sin = sin
    for i in range(iterations):
        # this call to local_sin requires a local lookup
        result += local_sin(i)

In [97]:
%timeit tight_loop_slow(10000000)

1.51 s ± 71.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%timeit tight_loop_fast(10000000)

1.45 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们可以看出，第二种方法比较快，虽然这种增益并不大。